# Lab 6 - Zmiana wartości pikseli – Metody na obrazie

1. `im.getpixel` i `im.putpixel` - do pojedynczych operacji na pikselach
2. `im.load()` - do szybszych operacji na wielu pikselach
3. `im.point` - do szybkiego zastosowania funkcji na wszystkich pikselach

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## 1. Funkcje wstawiania inicjałów

Implementujemy dwie wersje funkcji do wstawiania inicjałów:
- Używając getpixel/putpixel
- Używając metody load()

In [ ]:
def wstaw_inicjaly(obraz, inicjaly, m, n, kolor):
    """Wstawia inicjały w określonym kolorze na obraz używając getpixel/putpixel"""
    img_copy = obraz.copy()
    width, height = inicjaly.size
    
    for i in range(width):
        for j in range(height):
            if inicjaly.getpixel((i, j)) == 0:  # czarny piksel w inicjałach
                img_copy.putpixel((m + i, n + j), kolor)
    
    return img_copy

def wstaw_inicjaly_maska(obraz, inicjaly, m, n):
    """Tworzy negatyw w miejscach czarnych pikseli inicjałów"""
    img_copy = obraz.copy()
    width, height = inicjaly.size
    
    for i in range(width):
        for j in range(height):
            if inicjaly.getpixel((i, j)) == 0:
                pixel = img_copy.getpixel((m + i, n + j))
                if isinstance(pixel, int):
                    img_copy.putpixel((m + i, n + j), 255 - pixel)
                else:
                    img_copy.putpixel((m + i, n + j), tuple(255 - x for x in pixel))
    
    return img_copy

## 2. Wersje funkcji używające load()

Te same funkcje, ale wykorzystujące szybszą metodę load()

In [ ]:
def wstaw_inicjaly_load(obraz, inicjaly, m, n, kolor):
    """Wstawia inicjały używając metody load()"""
    img_copy = obraz.copy()
    width, height = inicjaly.size
    pix = img_copy.load()
    init_pix = inicjaly.load()
    
    for i in range(width):
        for j in range(height):
            if init_pix[i, j] == 0:
                pix[m + i, n + j] = kolor
    
    return img_copy

def wstaw_inicjaly_maska_load(obraz, inicjaly, m, n):
    """Tworzy negatyw używając metody load()"""
    img_copy = obraz.copy()
    width, height = inicjaly.size
    pix = img_copy.load()
    init_pix = inicjaly.load()
    
    for i in range(width):
        for j in range(height):
            if init_pix[i, j] == 0:
                pixel = pix[m + i, n + j]
                if isinstance(pixel, int):
                    pix[m + i, n + j] = 255 - pixel
                else:
                    pix[m + i, n + j] = tuple(255 - x for x in pixel)
    
    return img_copy

## 3. Transformacje obrazu

Implementacja różnych transformacji obrazu używając metody point()

In [ ]:
def kontrast(obraz, wsp_kontrastu):
    """Zmienia kontrast obrazu"""
    img_copy = obraz.copy()
    mn = ((255 + wsp_kontrastu) / 255) ** 2
    return img_copy.point(lambda i: 128 + (i - 128) * mn)

def transformacja_logarytmiczna(obraz):
    """Aplikuje transformację logarytmiczną"""
    img_copy = obraz.copy()
    return img_copy.point(lambda i: 255 * np.log(1 + i / 255))

def transformacja_gamma(obraz, gamma):
    """Aplikuje korekcję gamma"""
    img_copy = obraz.copy()
    return img_copy.point(lambda i: (i / 255) ** (1 / gamma) * 255)

## 4. Porównanie metod dodawania wartości do obrazu

Demonstracja różnicy między dodawaniem wartości do tablicy uint8 a metodą point.
    
Metoda z użyciem uint8 daje inny wynik, ponieważ typ uint8 używa arytmetyki modulo 256, więc wartości > 255 są "zawijane". Np. 200 + 100 = 44 (w uint8), bo (300 % 256 = 44).
    
Natomiast metoda point zachowuje wartości w pełnym zakresie przed końcową konwersją do uint8, więc wartości > 255 są po prostu ucinane do 255.

dodaj_wartosc_jak_point():

Funkcja działająca na tablicy obrazu, dająca taki sam efekt jak obraz.point(lambda i: i + wartosc)
    
Aby uzyskać ten sam efekt:
1. Konwertujemy do float64 aby uniknąć przepełnienia
2. Dodajemy wartość
3. Używamy clip aby ograniczyć wartości do zakresu [0, 255]
4. Konwertujemy z powrotem do uint8

In [ ]:
def zadanie5_porownanie_metod(obraz):
    """Porównanie różnych metod dodawania wartości do obrazu"""
    # Metoda 1 - nieprawidłowa (z użyciem uint8)
    T = np.array(obraz, dtype='uint8')
    T += 100
    obraz_nieprawidlowy = Image.fromarray(T, "RGB")
    
    # Metoda 2 - point
    obraz_point = obraz.point(lambda i: i + 100)
    
    # Porównanie wyników
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.imshow(obraz)
    plt.title("Oryginalny")
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(obraz_nieprawidlowy)
    plt.title("Metoda z uint8\n(nieprawidłowa)")
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(obraz_point)
    plt.title("Metoda point\n(prawidłowa)")
    plt.axis('off')
    
    plt.savefig("fig5_porownanie.png")
    plt.close()
    
    return obraz_nieprawidlowy, obraz_point

def dodaj_wartosc_jak_point(obraz, wartosc):
    """Prawidłowa implementacja dodawania wartości do obrazu"""
    T = np.array(obraz, dtype='float64')
    T += wartosc
    T = np.clip(T, 0, 255)
    return Image.fromarray(T.astype('uint8'), "RGB")

## 5. Główna funkcja testująca

Funkcja main() testująca wszystkie zaimplementowane funkcje

In [ ]:
def main():
    # Wczytanie obrazów
    obraz = Image.open("baby_yoda.jpg")
    inicjaly = Image.open("inicjaly.jpg").convert('L')
    
    # Test funkcji wstawiania inicjałów
    img_width, img_height = obraz.size
    init_width, init_height = inicjaly.size
    
    obraz1 = wstaw_inicjaly(obraz, inicjaly, 
                           img_width - init_width, 
                           img_height - init_height, 
                           (255, 0, 0))
    obraz1.save("obraz1.png")
    
    obraz2 = wstaw_inicjaly_maska(obraz, inicjaly,
                                 (img_width - init_width) // 2,
                                 (img_height - init_height) // 2)
    obraz2.save("obraz2.png")
    
    # Test transformacji
    for wsp in [20, 50, 80]:
        kontrast(obraz, wsp).save(f"kontrast_{wsp}.png")
    
    transformacja_logarytmiczna(obraz).save("logarytmiczna.png")
    
    for gamma in [0.5, 1.0, 2.0]:
        transformacja_gamma(obraz, gamma).save(f"gamma_{gamma}.png")
    
    # Test metod dodawania wartości
    zadanie5_porownanie_metod(obraz)
    dodaj_wartosc_jak_point(obraz, 100).save("dodawanie_prawidlowe.png")

if __name__ == "__main__":
    main()

## Wnioski

1. **Metody manipulacji pikselami**:
   - `getpixel/putpixel` są najwolniejsze, ale dobre do pojedynczych zmian
   - `load()` jest znacznie szybszy przy wielu operacjach
   - `point` jest najszybszy do jednolitych transformacji

2. **Transformacje obrazu**:
   - Kontrast: wartości `wsp_kontrastu` wpływają na rozpiętość tonalną obrazu
   - Transformacja logarytmiczna: uwydatnia szczegóły w ciemnych obszarach
   - Korekcja gamma: pozwala na nieliniową korektę jasności

3. **Dodawanie wartości do obrazu**:
   - Użycie `uint8` prowadzi do nieprawidłowych wyników przez zawijanie wartości
   - Prawidłowa implementacja wymaga użycia typu `float64` i funkcji `clip`
   - Metoda `point` automatycznie obsługuje przekroczenie zakresu wartości